# Example usage

To use `linreg_ally` in a project:

In [ ]:
import linreg_ally

print(linreg_ally.__version__)

In [ ]:
# Imports

In [ ]:
# Paramveer - EDA

In [ ]:
# Alex - VIF 

In [ ]:
# Cheng - model fitting

In [ ]:
# Merari - plot